In [1]:
import numpy as np
import pandas as pd

from IPython.display import clear_output
from geopy.geocoders import Nominatim

In [2]:
train = pd.read_csv('../data/aux_minus_bad_coord_train.csv')

##### Se rellenan las ciudades para coordenadas válidadas. Primero se ubican estos casos.

In [24]:
indice = train.loc[(train['ciudad'].isnull()) & (~train['lat'].isnull())].index
train.loc[indice, :]

,ciudad,provincia,lat,lng
5334,NaN,Querétaro,20.643900,-100.399000
6239,NaN,Guanajuato,20.694400,-101.362000
8093,NaN,Edo. de México,19.603175,-99.228323
10221,NaN,Distrito Federal,19.365011,-99.244550
10725,NaN,Querétaro,20.591300,-100.349000
...,...,...,...,...
234214,NaN,NaN,20.675822,-100.458423
236923,NaN,San luis Potosí,20.523091,-100.425349
237457,NaN,Morelos,18.966600,-99.215900
237798,NaN,NaN,19.342322,-99.061227


In [12]:
geolocator = Nominatim(user_agent="my-application")

In [33]:
%%time
ini = 0
fin = indice.size
i=1
for index in indice[ini:fin] :
    clear_output(wait=True)
    location = geolocator.reverse(str(train.loc[index, 'lat'])+','+str(train.loc[index, 'lng']))
    diccionario = location.raw['address']
    data = diccionario.get("city", np.NaN)
    train.loc[index, 'ciudad'] = data
    print(i, 'de', fin - ini)
    i = i + 1

160 de 160
CPU times: user 2.16 s, sys: 373 ms, total: 2.53 s
Wall time: 2min 48s


##### Aquí se ubica el primer índice que no completa ciudad, luego de una excepción.

In [34]:
train.loc[(train['ciudad'].isnull()) & (~train['lat'].isnull())].head(3)

,ciudad,provincia,lat,lng
10221,NaN,Distrito Federal,19.365011,-99.244550
13362,NaN,Distrito Federal,19.359200,-99.140700
18785,NaN,NaN,19.391313,-99.179622


In [36]:
train.loc[(train['ciudad'].isnull()) & (~train['lat'].isnull())].shape

(50, 4)

##### Faltan llenar 50 casos. Se hace manualmente.

In [37]:
train.loc[indice, :].isnull().sum()

ciudad       50
provincia    37
lat           0
lng           0
dtype: int64

##### Cuando sale un error por acceso a índice en 0, quiere decir que se terminó la busqueda con las condiciones pedidas.

In [215]:
display(train.loc[(train['ciudad'].isnull()) & (~train['lat'].isnull())].head(1))
indice_aux = train.loc[(train['ciudad'].isnull()) & (~train['lat'].isnull())].index[0]

print(str(train.loc[indice_aux, 'lat'])+','+str(train.loc[indice_aux, 'lng']))

,ciudad,provincia,lat,lng


IndexError: index 0 is out of bounds for axis 0 with size 0

In [214]:
train.loc[indice_aux, 'ciudad'] = 'Tlalpan'
train.loc[indice_aux, :]

ciudad                Tlalpan
provincia    Distrito Federal
lat                   19.2993
lng                  -99.1169
Name: 238226, dtype: object

# Para chequear las ciudades que ya están en el DataFrame

In [50]:
ciudades = train['ciudad'].unique()

In [211]:
'Cuernavaca' in ciudades

True

### Ciudades comunes

- Álvaro Obregón
- Benito Juárez
- Tlajomulco de Zúñiga (Jalisco: Palomar, San Agustín)
- Melchor Ocampo
- Naucalpan de Juárez (Estado de México)
- Santiago de Querétaro
- Querétaro
- Coatepec
- Villa de Zaachila (Fraccionamiento Real del Valle)
- San Luis Potosí
- Santa María del Oro
- Tultitlán (Fuentes del Valle)
- Tlalpan
- Boca del Río
- Corregidora (Fraccionamiento Vista Real)
- Colón (Querétaro)
- Atizapán de Zaragoza (Estado de México)
- El Marqués (Querétaro)
- Centro (Ciudad de México, Villa Verdún)
- Zapopan (Jalisco, Santa Ana Tepetitlán)
- Soledad de Graciano Sánchez (San Luis Potosí)
- Villahermosa (Tabasco)
- Cuautlancingo (Puebla)
- Venustiano Carranza (Ciudad de México)
- San Pedro Mixtepec (Oaxaca, La Alejandría)
- Zihuatanejo de Azueta
- Cuernavaca
- Tlalnepantla de Baz
- Coyoacán

##### Se chequean casos nulos para las condiciones anteriores.

In [217]:
train.loc[(train['ciudad'].isnull()) & (~train['lat'].isnull())].shape

(0, 4)

##### Ya todas las coordenadas vñalidas tienen ciudades.

In [234]:
train.isnull().sum()

ciudad          212
provincia       155
lat          124129
lng          124129
dtype: int64

# Ahora hay que rellenar las provincias faltanes para coordenadas válidas

In [235]:
train.loc[(train['provincia'].isnull()) & (~train['lat'].isnull())].shape

(37, 4)

In [236]:
indice2 = train.loc[(train['provincia'].isnull()) & (~train['lat'].isnull())].index
train.loc[indice2, :].shape

(37, 4)

# Son 37 provincias

In [237]:
%%time
ini = 0
fin = indice2.size
i=1
for index in indice[ini:fin] :
    clear_output(wait=True)
    location = geolocator.reverse(str(train.loc[index, 'lat'])+','+str(train.loc[index, 'lng']))
    diccionario = location.raw['address']
    data = diccionario.get("state", np.NaN)
    train.loc[index, 'provincia'] = data
    print(i, 'de', fin - ini)
    i = i + 1

37 de 37
CPU times: user 439 ms, sys: 49.4 ms, total: 488 ms
Wall time: 38.8 s


In [239]:
train.loc[(train['provincia'].isnull()) & (~train['lat'].isnull())].shape

(27, 4)

# Quedan 27 Provincias, se completan manualmente

In [242]:
indice3 = train.loc[(train['provincia'].isnull()) & (~train['lat'].isnull())].index
indice3.size

27

##### Cuando sale un error por acceso a índice en 0, quiere decir que se terminó la busqueda con las condiciones pedidas.

In [305]:
display(train.loc[(train['provincia'].isnull()) & (~train['lat'].isnull())].head(1))
indice_aux = train.loc[(train['provincia'].isnull()) & (~train['lat'].isnull())].index[0]

print(str(train.loc[indice_aux, 'lat'])+','+str(train.loc[indice_aux, 'lng']))

,ciudad,provincia,lat,lng


IndexError: index 0 is out of bounds for axis 0 with size 0

In [304]:
train.loc[indice_aux, 'provincia'] = 'Ciudad de México'
train.loc[indice_aux, :]

ciudad             Iztapalapa
provincia    Ciudad de México
lat                   19.3423
lng                  -99.0612
Name: 237798, dtype: object

### Para chequear las provincias que ya están en el DataFrame

In [244]:
provincias = train['provincia'].unique()

In [281]:
'Edo. de México' in provincias

True

### Provincias comunes

- Querétaro
- San Luis Potosí
- Ciudad de México

In [306]:
train.isnull().sum()

ciudad          212
provincia       118
lat          124129
lng          124129
dtype: int64

Se verifican las provincias

In [5]:
train['provincia'].value_counts()

Distrito Federal         58786
Edo. de México           41604
Jalisco                  21238
Querétaro                17000
Nuevo León               15324
Puebla                   10421
San luis Potosí           8440
Yucatán                   7928
Morelos                   7337
Veracruz                  5762
Quintana Roo              4756
Chihuahua                 4590
Coahuila                  3695
Baja California Norte     3220
Sonora                    2988
Guanajuato                2860
Guerrero                  2678
Hidalgo                   2521
Michoacán                 2471
Tamaulipas                2303
Durango                   2275
Sinaloa                   1806
Aguascalientes            1753
Baja California Sur       1700
Nayarit                   1352
Chiapas                   1126
Colima                    1008
Tabasco                    994
Tlaxcala                   839
Oaxaca                     711
Campeche                   263
Zacatecas                   94
San Luis

##### Se realizan alguno cambios en los nombres de las provincias.
- Distrito Federal $\rightarrow$ Ciudad de México
- San luis Potosí $\rightarrow$ San Luis Potosí
- Edo. de México $\rightarrow$ Estado de México

In [6]:
diccionario = {
    'Distrito Federal':'Ciudad de México',
    'San luis Potosí':'San Luis Potosí',
    'Edo. de México':'Estado de México'
}
diccionario

{'Distrito Federal': 'Ciudad de México',
 'San luis Potosí': 'San Luis Potosí',
 'Edo. de México': 'Estado de México'}

In [8]:
train['provincia'].replace(diccionario, inplace=True)

In [10]:
train['provincia'].value_counts()

Ciudad de México         58801
Estado de México         41609
Jalisco                  21238
Querétaro                17000
Nuevo León               15324
Puebla                   10421
San Luis Potosí           8459
Yucatán                   7928
Morelos                   7337
Veracruz                  5762
Quintana Roo              4756
Chihuahua                 4590
Coahuila                  3695
Baja California Norte     3220
Sonora                    2988
Guanajuato                2860
Guerrero                  2678
Hidalgo                   2521
Michoacán                 2471
Tamaulipas                2303
Durango                   2275
Sinaloa                   1806
Aguascalientes            1753
Baja California Sur       1700
Nayarit                   1352
Chiapas                   1126
Colima                    1008
Tabasco                    994
Tlaxcala                   839
Oaxaca                     711
Campeche                   263
Zacatecas                   94
Name: pr

##### Son 31 estados y la Ciudad de México, más los NaN, tendría que haber 33 valores únicos.

In [12]:
print(train['provincia'].unique())
print(train['provincia'].unique().size)

['Ciudad de México' 'Jalisco' 'Estado de México' 'Oaxaca' 'Quintana Roo'
 'Colima' 'San Luis Potosí' 'Nuevo León' 'Querétaro' 'Tamaulipas' 'Puebla'
 'Yucatán' 'Morelos' 'Guerrero' 'Tabasco' 'Guanajuato' 'Hidalgo'
 'Veracruz' 'Chihuahua' 'Aguascalientes' 'Sonora' 'Michoacán'
 'Baja California Norte' 'Baja California Sur' 'Coahuila' 'Durango'
 'Sinaloa' 'Chiapas' 'Nayarit' 'Tlaxcala' 'Campeche' 'Zacatecas' nan]
33


##### Ahora se rellenan las coordenadas según ciudades con las coordenadas obtenidas previamente, por ciudad.

In [19]:
coord_ciudad = pd.read_csv('../data/info_coord_ciudades.csv', index_col='ciudades')

In [20]:
coord_ciudad.head(5)

,lat,lng
ciudades,,
Querétaro,20.854258,-99.847560
Benito Juárez,20.816967,-98.178268
Zapopan,20.703205,-103.426124
San Luis Potosí,22.500000,-100.500000
Mérida,20.967076,-89.623740


In [6]:
train.isnull().sum()

ciudad          212
provincia       118
lat          124129
lng          124129
dtype: int64

In [9]:
train.loc[(train['lat'].isnull()) & (train['lng'].isnull())].isnull().sum()

ciudad          212
provincia       118
lat          124129
lng          124129
dtype: int64

##### Las coordenadas fantantes vienen de a pares.
##### Se ubican las filas con coordenadas faltantes y ciudades válidas.

In [17]:
indice4 = train.loc[(train['lat'].isnull()) & \
                    (train['lng'].isnull()) | \
                    (train['ciudad']).isnull()].dropna(subset=['ciudad']).index
# Se limpiaron las ciudades nulas
indice4.size

123917

# Se reemplazan los nulos con las coordenas por ciudad
##### Con esto se ocupan  las latitudes y longitudes de acuerdo a las ciudades no nulas (se usan los índices de ciudades no nulas). Para localizar la ciudad en el archivo de coordenadas, se busca la ciudad del train para ubicar el índice.

In [22]:
%%time
i=1
for index in indice4 :
    clear_output(wait=True)
    train.loc[index, 'lat'] = coord_ciudad.loc[train.loc[index, 'ciudad'], 'lat']
    train.loc[index, 'lng'] = coord_ciudad.loc[train.loc[index, 'ciudad'], 'lng']
    print(i, 'de', indice4.size)
    i = i + 1

123917 de 123917
CPU times: user 13min 24s, sys: 54 s, total: 14min 18s
Wall time: 12min 31s


In [23]:
train.isnull().sum()

ciudad       212
provincia    118
lat          212
lng          212
dtype: int64

In [33]:
train.loc[(train['lat'].isnull()) & \
          (train['lng'].isnull()) & \
          (train['ciudad'].isnull())]

,ciudad,provincia,lat,lng
630,NaN,Nuevo León,NaN,NaN
5307,NaN,San Luis Potosí,NaN,NaN
8385,NaN,Chihuahua,NaN,NaN
10337,NaN,Chihuahua,NaN,NaN
10555,NaN,NaN,NaN,NaN
...,...,...,...,...
234536,NaN,Nuevo León,NaN,NaN
236052,NaN,NaN,NaN,NaN
236088,NaN,Chihuahua,NaN,NaN
236452,NaN,San Luis Potosí,NaN,NaN


##### Ya no hay ciudades válidas sin coordenadas.

##### Ahora se completan las coordenadas de las provincias válidas.

In [35]:
train.loc[(train['lat'].isnull()) & \
          (train['lng'].isnull()) & \
          (train['ciudad'].isnull()) & \
          (~train['provincia'].isnull())]

,ciudad,provincia,lat,lng
630,NaN,Nuevo León,NaN,NaN
5307,NaN,San Luis Potosí,NaN,NaN
8385,NaN,Chihuahua,NaN,NaN
10337,NaN,Chihuahua,NaN,NaN
10820,NaN,Jalisco,NaN,NaN
...,...,...,...,...
231193,NaN,Guerrero,NaN,NaN
234167,NaN,Chihuahua,NaN,NaN
234536,NaN,Nuevo León,NaN,NaN
236088,NaN,Chihuahua,NaN,NaN


##### Se pueden completar 94 coordenadas con las de las provincias.

In [41]:
coord_provincia = pd.read_csv('../features/info_coord_provincias.csv', index_col='provincias')

coord_provincia.head(5)

,lat,lng
provincias,,
Ciudad de México,19.432601,-99.133342
Estado de México,19.483945,-99.689972
Jalisco,20.333333,-103.666667
Querétaro,20.854258,-99.847560
Nuevo León,26.238436,-99.887300


In [42]:
train.isnull().sum()

ciudad       212
provincia    118
lat          212
lng          212
dtype: int64

In [43]:
train.loc[(train['lat'].isnull()) & (train['lng'].isnull())].isnull().sum()

ciudad       212
provincia    118
lat          212
lng          212
dtype: int64

##### Las coordenadas fantantes vienen de a pares.
##### Se ubican las filas con coordenadas faltantes y provincias válidas.

In [45]:
indice5 = train.loc[(train['lat'].isnull()) & \
          (train['lng'].isnull()) & \
          (train['ciudad'].isnull()) & \
          (~train['provincia'].isnull())].index
indice5.size

94

# Se reemplazan los nulos con las coordenas por provincia
##### Con esto se ocupan  las latitudes y longitudes de acuerdo a las provincias no nulas (se usan los índices de provincias no nulas). Para localizar la provincia en el archivo de coordenadas, se busca la provincia del train para ubicar el índice.

In [46]:
%%time
i=1
for index in indice5 :
    clear_output(wait=True)
    train.loc[index, 'lat'] = coord_provincia.loc[train.loc[index, 'provincia'], 'lat']
    train.loc[index, 'lng'] = coord_provincia.loc[train.loc[index, 'provincia'], 'lng']
    print(i, 'de', indice5.size)
    i = i + 1

94 de 94
CPU times: user 557 ms, sys: 53.8 ms, total: 611 ms
Wall time: 536 ms


In [47]:
train.isnull().sum()

ciudad       212
provincia    118
lat          118
lng          118
dtype: int64

### Se genera un Dataframe con las modas de ciudades por provincias.

In [63]:
moda_train = train.groupby('provincia').agg(lambda x:x.value_counts().index[0])
moda_train.drop(columns=['lat', 'lng'], inplace=True)
moda_train

,ciudad
provincia,
Aguascalientes,Aguascalientes
Baja California Norte,Tijuana
Baja California Sur,La Paz
Campeche,Campeche
Chiapas,Tuxtla Gutiérrez
Chihuahua,Chihuahua
Ciudad de México,Benito Juárez
Coahuila,Saltillo
Colima,Colima


##### Se buscan las ciudades nulas, con coordenadas y provincias válidas.

In [67]:
indice6 = train.loc[(train['ciudad'].isnull()) & \
                    (~train['lat'].isnull()) & \
                    (~train['lng'].isnull()) & \
                    (~train['provincia'].isnull())].index
indice6.size

94

In [68]:
%%time
i=1
for index in indice6 :
    clear_output(wait=True)
    train.loc[index, 'ciudad'] = moda_train.loc[train.loc[index, 'provincia'], 'ciudad']
    print(i, 'de', indice6.size)
    i = i + 1

94 de 94
CPU times: user 903 ms, sys: 68.8 ms, total: 972 ms
Wall time: 902 ms


In [69]:
train.isnull().sum()

ciudad       118
provincia    118
lat          118
lng          118
dtype: int64

##### Ahora se rellena con la moda de provincia, ciudad y las coordenadas que corresponda a esa ciudad.

In [74]:
provincia_moda = train['provincia'].mode()[0]
provincia_moda

'Ciudad de México'

In [75]:
ciudad_moda = moda_train.loc[train['provincia'].mode()[0], 'ciudad']
ciudad_moda

'Benito Juárez'

In [80]:
latitud_moda = coord_ciudad.loc[ciudad_moda, 'lat']
longitud_moda = coord_ciudad.loc[ciudad_moda, 'lng']
print(longitud_moda)
print(latitud_moda)

-98.1782680664942
20.816966600000004


In [81]:
train['provincia'].fillna(provincia_moda, inplace=True)
train['ciudad'].fillna(ciudad_moda, inplace=True)
train['lat'].fillna(latitud_moda, inplace=True)
train['lng'].fillna(longitud_moda, inplace=True)

In [82]:
train.isnull().sum()

ciudad       0
provincia    0
lat          0
lng          0
dtype: int64

In [83]:
train.shape

(240000, 4)

In [84]:
train

,ciudad,provincia,lat,lng
0,Benito Juárez,Ciudad de México,20.816967,-98.178268
1,La Magdalena Contreras,Ciudad de México,19.310205,-99.227655
2,Tonalá,Jalisco,20.618521,-103.222736
3,Zinacantepec,Estado de México,19.301890,-99.688015
4,Zapopan,Jalisco,20.703205,-103.426124
...,...,...,...,...
239995,Zinacantepec,Estado de México,19.203879,-99.814605
239996,Toluca,Estado de México,19.294665,-99.692916
239997,Benito Juárez,Ciudad de México,20.816967,-98.178268
239998,Iztapalapa,Ciudad de México,19.366651,-99.082246


In [85]:
train.to_csv('../data/features/ciudad_prov_lat_lng_V2_train.csv', index=False)

In [86]:
prueba = pd.read_csv('../data/features/ciudad_prov_lat_lng_V2_train.csv')
prueba

,ciudad,provincia,lat,lng
0,Benito Juárez,Ciudad de México,20.816967,-98.178268
1,La Magdalena Contreras,Ciudad de México,19.310205,-99.227655
2,Tonalá,Jalisco,20.618521,-103.222736
3,Zinacantepec,Estado de México,19.301890,-99.688015
4,Zapopan,Jalisco,20.703205,-103.426124
...,...,...,...,...
239995,Zinacantepec,Estado de México,19.203879,-99.814605
239996,Toluca,Estado de México,19.294665,-99.692916
239997,Benito Juárez,Ciudad de México,20.816967,-98.178268
239998,Iztapalapa,Ciudad de México,19.366651,-99.082246
